# Rekurentne neuronske mreže

U ovoj svesci sumirani su osnovni koncepti vezani za rekturentne neuronske mreže i podrška torch biblioteke u radu sa njima.

<img src='assets/RNN.png'>
<p style='text-align: right; color: gray; font-size: 10px;'> Slika je pozajmljena iz prezentacije Recurrent Neural Networks Momčila Vasiljevića, MDCS </p>

Ono što je zajedničko za sve rekurentne neuronske mreže je da obrađuju sekvence podataka element po element. Na primer, mogu se obrađivati reči teksta sa ciljem da se predvidi naredna reč ili blok reči (takozvani zadatak dopune teksta, engl. text completition), cene akcija na berzi sa ciljem da se predvidi naredna vrednost i slično. Zato se o rekurentnim neuronskim mrežama obično priča sa aspekta vremenske dimenzije: u obradi u trenutku $t$ uzima se u obzir ulaz mreže $x_t$ i stanje mreže $s_{t-1}$ kojim se sumira ono što je mreža do tog trenutka obradila kako bi se generisao izlaz $o_t$. 

In [1]:
import torch
import numpy as np

SEED = 7
# za determinističko izvršavanje sveske:
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

## Jednostavna rekurentna neuronska mreža

Jednostavna rekurentna neuronska mreža je obična potpuno povezana mreža koja u trenutku $t$ na osnovu ulaza $x_t$ i prethodnog izlaza $o_{t-1}$ odlučuje o novom izlazu $o_t$. <img src='assets/simple_RNN.png'>

Sledećim kodom opisan je jedan prolaz kroz ovako definisanu rekurentnu neuronsku mrežu.

Dužinu sekvence ćemo zadati promenljivom `timesteps`.

In [2]:
timesteps = 100

Ulazi u mrežu će nam biti vektori dužine `inuput_size`.  Dužina ovog vektora odgovara broju atributa (engl. features)  koje imamo na nivou pojedinačnih instanci. 

In [3]:
input_size = 32 

Matrica svih ulaza će biti dimenzija `timesteps` x `input_size`. Inicijalizovaćemo je nasumičnim vrednostima.
<img src='assets/sequence_data.png'>

In [4]:
inputs = np.random.random((timesteps, input_size))

Izlazi mreže će biti vektori dužine `output_size`. Sa `output_t_prev` ćemo obeležavati vrednost izlaza za trenutak `t-1`, a sa `output_t` vrednost izlaza za trenutak `t`. Na početku će vektor prethodnih izlaza biti vektor nula.

In [5]:
output_size = 64 

In [6]:
output_t_prev = np.zeros(output_size)

Ovu rekurentnu neuronsku mrežu karakteriše matrica `U` koji povezuje trenutni ulaz `x_t` sa trenutnim izlazom `o_t` i matrica `V` koja povezuje trenutni izlaz `o_t` sa prethodnim izlazom `o_t_prev`. Ove matrice ćemo nasumično generisati. Generisaćemo i nasumični vektor slobodnih članova `b`.

In [7]:
U = np.random.random((output_size, input_size))
V = np.random.random((output_size, output_size))

b = np.random.random(output_size)

<img src='assets/simple_RNN_unrolled.png'>

Jedan prolaz kroz mrežu u obradi sekvence `inputs` je opisan kodom ispod. 

In [8]:
# izlazi mreze
successive_outputs = []

# za svaki ulaz u sekvenci
for input_t in inputs:
    
    # kombinuje se tekuci ulaz input_t i prethodni izlaz output_t_prev i izračunava se novi izlaz
    output_t = np.tanh(U @ input_t + V @ output_t_prev + b)
    
    # trenutni izlaz mreze treba uzeti u obzir prilikom obrade sledeceg ulaza 
    output_t_prev = output_t.copy()    
    
    # cuvamo izlaz mreze
    successive_outputs.append(output_t)


Niz svih izlaza je dužine koja odgovara dužini sekvence, preciznije oblika `input_size` x `output_size`.

In [9]:
len(successive_outputs)

100

In [10]:
np.array(successive_outputs).shape

(100, 64)

## Rekurentna neuronska mreža sa prenosom skrivenog stanja

Kada se govori o rekurentnim neuronskim mrežama ponajčešće se misli na mreže koje imaju arhitekturu koja je prikazana na slici. 

<img src='assets/RNN_unfolded.png'>

Njome se u trenutku $t$ na osnovu ulaza $x_t$ i prethodnog stanja skrivenog sloja $h_{t-1}$ odlučuje o novom izlazu $o_t$.

Prvo ćemo ponoviti deklaracije promenljivih. Njihovo značenje je i u kontekstu ove arhitetkure identično prethodnim.

In [11]:
timestamps = 100
input_size = 32 
inputs = np.random.random((timestamps, input_size))
output_size = 64 

Veličinu skrivenog sloja mreže ćemo zadati dimenzijom `hidden_size`. Sa `h_t_prev` ćemo obeležavati vrednost skrivenog sloja za trenutak `t-1`, a sa `h_t` vrednost skrivenog sloja za trenutak `t`. Na početku treniranja `h_t_prev` će biti vektor nula.

In [12]:
hidden_size = 32

In [13]:
h_t_prev = np.zeros(hidden_size)

Ovu rekurentnu neuronsku mrežu karakteriše matrica `U` sloja koji povezuje trenutni ulaz `x_t` sa trenutnom vrednošću skrivenog sloja `h_t`, matrica `V` koja povezuje stanje skrivenog sloja iz prethodnog trenutka `h_t_prev` i tekuće stanje `h_t`, i matrica `W` sloja kojom se povezuje skriveno stanje `h_t` sa izlazom `o_t`. 

In [14]:
U = np.random.random((hidden_size, input_size))
V = np.random.random((hidden_size, hidden_size))
W = np.random.random((output_size, hidden_size))

# vektori slobodnih clanova 
b1 = np.random.random(hidden_size)
b2 = np.random.random(output_size)

Jedan prolaz kroz mrežu u obradi sekvence `inputs` je opisan kodom ispod. 

In [15]:
# izlazi mreze
successive_outputs = []

# za svaki ulaz u sekvenci
for input_t in inputs:
    
    # kombinuje se tekuci ulaz input_t i stanje skrivenog sloja mreze iz prethodnog koraka h_t_prev 
    # i izračunava se novo stanje skrivenog sloja 
    h_t = np.tanh(U @ input_t + V @ h_t_prev + b1)
    
    # izracuna se izlaz mreze
    output_t= np.tanh(W @ h_t + b2)

    # trenutne vrednosti skrivenog sloja mreze traba uzeti u obzir prilikom obrade sledeceg ulaza 
    h_t_prev = h_t.copy()    
    
    # cuvamo izlaz mreze
    successive_outputs.append(output_t)


Niz svih izlaza je dužine koja odgovara dužini sekvence, preciznije oblika `timestamps` x `output_size`.

In [16]:
len(successive_outputs)

100

In [17]:
np.array(successive_outputs).shape

(100, 64)

## LSTM

Svojom specifičnom strukturom, LSTM (engl. Long-Short Term Memory) ćelije nude rešenje za praćenje dugoročnih zavisnosti na nivou sekvenci koje zbog problema isčezavajućih gradijenata (praktično) nije bilo moguće sa stanradnim RNN ćelijama. 

Uz vrednosti skrivenog sloja ove ćelije imaju svoje interno stanje, ulaznu kapiju (engl. input gate), kapiju za zaboravljanje (engl. forget gate) i izlaznu kapiju (engl. output gate) kojima, redom, mogu da kontrolišu prisutnost informacija iz prethodnog stanja, njihovu selekciju i generisanje vrednosti koje će dalje propagirati. 

<img src='assets/LSTM_cell_2.png'>

U trenutku $t$ na osnovu ulaza $x_t$, vrednosti skrivenog sloja $h_{t-1}$ i stanja $c_{t-1}$ prethodnog koraka obrade, treba odlučiti o novoj vrednosti skrivenog sloja $h_t$, stanja $c_t$ i izlaza $o_t$. 

Kapijom za zaboravljanje $f_t$ se kontroliše koliko informacija iz prethodnog stanja treba dalje propustiti. Na osnovu vrednosti matrica $W_f$ i $U_f$ i vektora slobodnih članova $b_f$ koje je karakterišu, izračunavaju se vrednosti vektora $f_t$ koje su u opsegu [0, 1]. Pokoordinatnim množenjem sa vrednošću $c_{t-1}$ dobijaju se filtrirane informacije stanja. Dalje se, na sličan način, izračunavaju vrednosti ulazne kapije $i_t$ i vrednosti izlazne kapije $o_t$. Ulaznom kapijom se filtrijaju informacije međustanja ćelije $\tilde{c_t}$ koje treba dodati informacijama koje se filtrijaju kapijom zaboravljanja kako bi se dobilo novo stanje $c_t$. Nova vrednost skrivenog stanja se dobija na osnovu vrednosti izlazne kapije $o_t$ i novog stanja ćelije $c_t$. 

<img src='assets/LSTM_math.png'>

In [18]:
timesteps = 100
input_size = 32
output_size = 64

In [19]:
# parametri za kapiju zaboravljanja
W_f = np.random.random((output_size, input_size))
U_f = np.random.random((output_size, output_size))
b_f = np.random.random((output_size,))

# parametri za ulazne kapije
W_i = np.random.random((output_size, input_size))
U_i = np.random.random((output_size, output_size))
b_i = np.random.random((output_size,))

# parametri za izlazne kapije
W_o = np.random.random((output_size, input_size))
U_o = np.random.random((output_size, output_size))
b_o = np.random.random((output_size,))

# parametri medjustanja
W_c = np.random.random((output_size, input_size))
U_c = np.random.random((output_size, output_size))
b_c = np.random.random((output_size,))

In [20]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [21]:
inputs = np.random.randn(timesteps, input_size)

# inicijalno stanje LSTM celije
c = np.zeros((output_size,))
h = np.zeros((output_size,))

# izlazi mreze
successive_outputs = []
for input in inputs:
    f = sigmoid(W_f @ input + U_f @ h + b_f)
    i = sigmoid(W_i @ input + U_i @ h + b_i)
    o = sigmoid(W_o @ input + U_o @ h + b_o)
    c_tilda = np.tanh(W_c @ input + U_c @ h + b_c)
    c = f * c + i * c_tilda
    h = o * np.tanh(c)
    successive_outputs.append(h)
    


## GRU ćelije

<img src='assets/GRU_cell.png'>

GRU (engl. Gated-Recurent Units) ćelije predstavljaju modifikaciju LSTM ćelija objedinjavanjem kapije ulaza i kapije zaboravljanja u jednu kapiju ažuriranja.

U praksi rad sa GRU ćelijama vodi do bržeg treniranja. 

In [22]:
timesteps = 100
input_size = 32
output_size = 64

In [23]:
# parametri za kapiju reset
W_r = np.random.random((output_size, input_size))
U_r = np.random.random((output_size, output_size))
b_r = np.random.random((output_size,))

# parametri za kapiju ažuriranja
W_z = np.random.random((output_size, input_size))
U_z = np.random.random((output_size, output_size))
b_z = np.random.random((output_size,))

# parametri medjustanja
W_h = np.random.random((output_size, input_size))
U_h = np.random.random((output_size, output_size))
b_h = np.random.random((output_size,))


In [24]:
inputs = np.random.randn(timesteps, input_size)

# inicijalno stanje GRU celije
h = np.zeros((output_size,))

# izlazi mreze
successive_outputs = []
for input in inputs:
    z = sigmoid(W_z @ input + U_z @ h + b_z)
    r = sigmoid(W_r @ input + U_r @ h + b_r)
    h_tilda = np.tanh(W_h @ input + U_h @ (r * h) + b_h)
    h = (1 - z) * h + z * h_tilda
    successive_outputs.append(h)
    


## Odgovarajuća torch podrška

U biblioteci `torch`, podrška je omogućena za rad sa jednostavnim rekurentnim neuronskim mrežama (RNN), LSTM mrežama i GRU slojevima. Za izračunavanja koja smo demonstrirali u uvodnim primerima unutar for petlje, biblioteka `torch` koristi klase `RNNCell`, `LSTMCell` i `GRUCell`. Ove klase su dizajnirane da obrađuju podatke organizovane u pakete sa strukturom `(timesteps, batch_size, N)`.

In [25]:
import torch
import torch.nn as nn

In [26]:
timesteps = 100
input_size = 32
output_size = 64
batch_size = 16

In [27]:
# inicijalizacija početnih stanja, često se koristi 0
h = torch.zeros(batch_size, output_size)
c = torch.zeros(batch_size, output_size)

In [28]:
# Primer koriscenja LSTMCell klase
lstm_cell = nn.LSTMCell(input_size=input_size, hidden_size=output_size) 
inputs = torch.randn(timesteps, batch_size, input_size) 

output = []
for input_t in inputs:
    h, c = lstm_cell(input_t, (h, c))
    output.append(h)
output = torch.stack(output, dim=0)

Pored ovih osnovnih klasa, modul `torch.nn` u biblioteci `torch` takođe uključuje klase `RNN`, `LSTM` i `GRU`. Ove klase omogućavaju izračunavanje svih skrivenih stanja sekvence unutar `forward` metoda, a takođe pružaju opciju za povezivanje višeslojnih rekurentnih neuronskih mreža, kako je ilustrovano na slici ispod.

<img src='assets/stacked_RNN.png'>

Broj slojeva može se kontrolisati pomoću parametra `num_layers` u konstruktorima ovih klasa. Dodatno, u konstruktoru je moguće izmeniti podrazumevanu strukturu paketića sa `(timesteps, batch_size, N)` u `(batch_size, timesteps, N)` aktiviranjem opcije `batch_first=True`.

Svaka od klasa `RNN`, `LSTM`, i `GRU` pruža dva rezultata nakon obrade podataka. Prvi rezultat predstavljaju izlazi iz poslednjeg sloja mreže (označeni kao $h_{10}$ i $h_{11}$ na slici), dok drugi uključuje skrivena stanja svakog sloja na kraju sekvence, što je na slici prikazano plavom bojom. Detaljnije objašnjenje i primere za svaku od ovih klasa možete pronaći u narednim odeljcima.

### RNN klasa

U [zvaničnoj dokumentaciji](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) možete videti da ova klasa vrši neznatno izmenjen račun u odnosu na metod koji smo opisali u uvodu. 

$h_t = tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{t-1} + b_{hh})$

Ovo nema uticaj na definiciju modela, već samo povećava broj parametara koje on ima. Ovo je urađeno zbog biblioteke cuDNN koju torch koristi za ubrzavanje ovih izračunavanja. Slične razlike će biti prisutne i u svim ostalim slojevima.

In [29]:
timesteps = 100
input_size = 32
output_size = 64
batch_size = 16

In [30]:
import torch.nn as nn
model = nn.RNN(input_size, hidden_size=output_size, num_layers=1, batch_first=True)

In [31]:
model

RNN(32, 64, batch_first=True)

Broj parametara napravljenog modela je $32 \cdot 64 + 64 + 64 \cdot 64 + 64 = 6272$ 

In [32]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

6272


RNN sloj daje dva izlaza: prvi je tenzor koji sadrži izlaze iz poslednjeg sloja mreže. Kada je parametar `batch_first` postavljen na `True`, ovaj tenzor će imati oblik `(batch_size, timesteps, hidden_size)`. Drugi izlaz predstavlja skrivena stanja posle obrade poslednjeg elementa u sekvenci i ima oblik `(num_layers, batch_size, hidden_size)`.

In [33]:
inputs = torch.randn(batch_size, timesteps, input_size)
model = nn.RNN(input_size, hidden_size=output_size, num_layers=3, batch_first=True)
output, h_last = model(inputs)

In [34]:
print(f"output: {output.shape}")
print(f"h_last: {h_last.shape}")

output: torch.Size([16, 100, 64])
h_last: torch.Size([3, 16, 64])


### LSTM klasa

Izračunavanja koja ova klasa vrši su sledeća:

$i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi})$

$f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf})$

$o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho})$

$g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg})$

$c_t = f_t \circ c_{t-1} + i_t \circ g_t$

$h_t = o_t \circ \tanh(c_t)$


In [35]:
timestamps = 100
input_size = 32 
output_size = 64

In [36]:
model = nn.LSTM(input_size=input_size, hidden_size=output_size)
model

LSTM(32, 64)

Svaka od matrica $W_{i\_}$ je dimenzije `output_size` x `input_size`, svaka od matrica  $W_{h\_}$ je dimenzije  `output_size` x `output_size` i svaki vektor slbodnih članova je dimenzije `output_size`. Ukupan broj parametara modela koji smo definisali je:

4 x 64 x 32 + 4 x 64 x 64 + 8 x 64 = 25088

In [37]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

25088


Slično RNN sloju, prvi izlaz LSTM sloja kada je parametar `batch_first=True` aktiviran biće tenzor oblika `(batch_size, timesteps, hidden_size)`. Međutim, budući da LSTM sloj upravlja sa dva tipa skrivenih stanja—stanje ćelije i skriveno stanje—drugi izlaz će biti uređeni par tenzora. Svaki od ovih tenzora imaće oblik `(num_layers, batch_size, hidden_size)`, pri čemu jedan predstavlja stanje ćelije, a drugi skriveno stanje, oba nakon obrade poslednjeg elementa u sekvenci.

In [38]:
model = nn.LSTM(input_size=input_size, hidden_size=output_size, num_layers=3, batch_first=True)
timestamps = 100
input_size = 32 
output_size = 64
batch_size = 128
input = torch.randn(batch_size, timestamps, input_size)
output, (h_n, c_n) = model(input)
print(f'output: {output.shape}')
print(f'   h_n: {h_n.shape}')
print(f'   c_n: {c_n.shape}')

output: torch.Size([128, 100, 64])
   h_n: torch.Size([3, 128, 64])
   c_n: torch.Size([3, 128, 64])


## GRU ćelije

Izračunavanja koja ova klasa vrši su sledeća:

$r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{t-1} + b_{hr})$

$z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{t-1} + b_{hz})$

$n_t = \tanh(W_{in} x_t + b_{in} + r_t \circ (W_{hn} h_{t-1} + b_{hn}))$

$h_t = (1 - z_t) \circ n_t + z_t \circ h_{t-1}$

In [39]:
from torch.nn import GRU

In [40]:
timestamps = 100
input_size = 32 
inputs = np.random.random((timestamps, input_size))
output_size = 64 

In [41]:
model = GRU(input_size=input_size, hidden_size=output_size, batch_first=True)

model

GRU(32, 64, batch_first=True)

Broj parametara modela je: 

3 x 32 x 64 + 3 x 64 x 64 + 6 x 64 = 18816

In [42]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

18816


Izlazi GRU sloja isti su kao i izlazi RNN sloja

In [43]:
inputs = torch.randn(batch_size, timesteps, input_size)
model = nn.RNN(input_size, hidden_size=output_size, num_layers=3, batch_first=True)
output, h_last = model(inputs)
print(f'output: {output.shape}')
print(f'h_last: {h_last.shape}')

output: torch.Size([128, 100, 64])
h_last: torch.Size([3, 128, 64])


### Za dalje istraživanje:
- [Ilustrovana pojašnjenja LSTM i GRU ćelija](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- Originalni rad u kojem su uvedene LSTM ćelije: [Long Short-Term Memory](http://www.bioinf.jku.at/publications/older/2604.pdf).
- Originalni red u kojem su uvedene GRU ćelije: [Learning Phrase Representations using RNN Encoder–Decoder
for Statistical Machine Translation](https://arxiv.org/pdf/1406.1078v3.pdf)
- [Coursera, Deep Learning Specialization - Sequence Modeling (part 5)](https://www.coursera.org/specializations/deep-learning)